In [2]:
from astropy.table import Table
import pandas as pd
from astropy.table.table import descr

def convert_fits(in_file, out_file):
    table = Table.read(in_file)
    
    new_table = Table()

    for col in table.columns.values():
        descriptor = descr(col)
        col_name = descriptor[0]
        col_shape = descriptor[2]
        if col_shape == (5,):
            data_t = col.data.T
            for index, band_char in enumerate('ugriz'):
                new_table.add_column(data_t[index], name=f"{col_name}_{band_char}")
        elif col_shape == ():
            new_table.add_column(col)

    new_table.to_pandas().to_parquet(out_file)

In [ ]:
from astropy.table import Table
import pandas as pd

def convert_fits_nested(in_file, out_file):
    table = Table.read(in_file)
    
    new_table = table.as_array().tolist()
    new_panda = pd.DataFrame(new_table, columns=table.columns.keys())

    new_panda.to_parquet(out_file)

In [4]:
import glob
import re
from tqdm import tqdm

files = glob.glob("/data3/epyc/data3/hipscat/raw/sdss/**.fits.gz")
files.sort()

for file in tqdm(files, bar_format='{l_bar}{bar:80}{r_bar}'):
    
    match = re.match(r".*(calibObj-.*-star).fits.gz", str(file))
    file_prefix = match.group(1)
#     print(file_prefix)
    output_filename = f"/data3/epyc/data3/hipscat/raw/sdss/parquet/{file_prefix}.parquet"
    
    convert_fits(file, output_filename)  


 66%|████████████████████████████████████████████████████▌                           | 3012/4584 [2:52:31<43:46,  1.67s/it]   IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|████████████████████████████████████████████████████████████████████████████████| 4584/4584 [4:45:40<00:00,  3.74s/it]  
